In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', -1)


import sklearn

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, make_scorer, mean_squared_error

<ipython-input-1-6676290e5037>:9: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [2]:
df = pd.read_csv('../data/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [3]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
len(df)

7043

In [5]:
df.head().T

,0,1,2,3,4
customerID,7590-VHVEG,5575-GNVDE,3668-QPYBK,7795-CFOCW,9237-HQITU
gender,Female,Male,Male,Male,Female
SeniorCitizen,0,0,0,0,0
Partner,Yes,No,No,No,No
Dependents,No,No,No,No,No
tenure,1,34,2,45,2
PhoneService,No,Yes,Yes,No,Yes
MultipleLines,No phone service,No,No,No phone service,No
InternetService,DSL,DSL,DSL,DSL,Fiber optic
OnlineSecurity,No,Yes,Yes,Yes,No


In [6]:
df.dtypes

customerID          object 
gender              object 
SeniorCitizen       int64  
Partner             object 
Dependents          object 
tenure              int64  
PhoneService        object 
MultipleLines       object 
InternetService     object 
OnlineSecurity      object 
OnlineBackup        object 
DeviceProtection    object 
TechSupport         object 
StreamingTV         object 
StreamingMovies     object 
Contract            object 
PaperlessBilling    object 
PaymentMethod       object 
MonthlyCharges      float64
TotalCharges        object 
Churn               object 
dtype: object

In [7]:
df['SeniorCitizen'].unique()

array([0, 1], dtype=int64)

In [8]:
#the to_numeric function raises an exception when it sees nonnumeric data (such as spaces), but we can make it skip these cases by specifying the errors='coerce' option.
#This way Pandas will replace all nonnumeric values with a NaN (not a number)
total_charges = pd.to_numeric(df['TotalCharges'], errors='coerce')

In [9]:
#To confirm that data contains nonnumeric characters, we can use isnull() function 
df[total_charges.isnull()][['customerID', 'TotalCharges']]

,customerID,TotalCharges
488,4472-LVYGI,
753,3115-CZMZD,
936,5709-LVOEQ,
1082,4367-NUYAO,
1340,1371-DWPAZ,
3331,7644-OMVMY,
3826,3213-VVOLG,
4380,2520-SGTTA,
5218,2923-ARZLG,
6670,4075-WKNIU,


In [10]:
#clean missing values

In [11]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'] = df['TotalCharges'].fillna(0)

In [12]:
#replacing all the column names with lower case and underscore seperators
df.columns = df.columns.str.lower().str.replace(' ','_')

In [13]:
#filtering for columns with the str dtype (need .index otherwise it ouputs values and just shows 'O')
str_cols = list(df.dtypes[df.dtypes == 'object'].index)
str_cols

['customerid',
 'gender',
 'partner',
 'dependents',
 'phoneservice',
 'multiplelines',
 'internetservice',
 'onlinesecurity',
 'onlinebackup',
 'deviceprotection',
 'techsupport',
 'streamingtv',
 'streamingmovies',
 'contract',
 'paperlessbilling',
 'paymentmethod',
 'churn']

In [14]:
#replace empty spaces with underscore in the string columns, there's prob a faster way than a for loop to do this but I'm just following the book
for col in str_cols:
    df[col] = df[col].str.lower().str.replace(' ', '_')

In [15]:
df['churn'].unique() #make 0 and 1

array(['no', 'yes'], dtype=object)

In [16]:
df['churn'] = (df['churn'] == 'yes').astype(int) #the result of the == operator: It's a Boolean series with True when the elements of the original series are "yes" and false otherwise

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
df_train_full, df_test = train_test_split(df, test_size=.2, random_state=1)

In [19]:
df_train_full.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
1814,5442-pptjy,male,0,yes,yes,12,yes,no,no,no_internet_service,no_internet_service,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,mailed_check,19.70,258.35,0
5946,6261-rcvns,female,0,no,no,42,yes,no,dsl,yes,yes,yes,yes,no,yes,one_year,no,credit_card_(automatic),73.90,3160.55,1
3881,2176-osjuv,male,0,yes,no,71,yes,yes,dsl,yes,yes,no,yes,no,no,two_year,no,bank_transfer_(automatic),65.15,4681.75,0
2389,6161-erdgd,male,0,yes,yes,71,yes,yes,dsl,yes,no,yes,yes,yes,yes,one_year,no,electronic_check,85.45,6300.85,0
3676,2364-ufrom,male,0,no,no,30,yes,no,dsl,yes,yes,no,yes,yes,no,one_year,no,electronic_check,70.40,2044.75,0


In [20]:
df_test.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
3381,8879-zkjof,female,0,no,no,41,yes,no,dsl,yes,no,yes,yes,yes,yes,one_year,yes,bank_transfer_(automatic),79.85,3320.75,0
6180,0201-mibol,female,1,no,no,66,yes,yes,fiber_optic,yes,no,no,no,yes,yes,two_year,yes,bank_transfer_(automatic),102.40,6471.85,0
4829,1600-dilpe,female,0,no,no,12,yes,no,dsl,no,no,no,no,no,no,month-to-month,yes,bank_transfer_(automatic),45.00,524.35,0
3737,8601-qacrs,female,0,no,no,5,yes,yes,dsl,no,no,no,no,no,no,month-to-month,yes,mailed_check,50.60,249.95,1
4249,7919-zodzz,female,0,yes,yes,10,yes,no,dsl,no,yes,yes,no,no,yes,one_year,yes,mailed_check,65.90,660.05,0


In [21]:
df_train, df_val = train_test_split(df_train_full, test_size=.25, random_state=11)
y_train = df_train['churn'].values #takes the column with the target variable and saves it outside the DataFrame
y_val = df_val['churn'].values #takes the column with the target variable and saves it outside the DataFrame

del df_train['churn'] #deletes the churn col from the df to prevent using the churn variable as a feature
del df_val['churn'] #deletes the churn col from the df to prevent using the churn variable as a feature

In [23]:
df_train_full.isnull().sum()

customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

In [24]:
df_train_full['churn'].value_counts()

0    4113
1    1521
Name: churn, dtype: int64

In [34]:
1521/5634 #about 27% of users stopped using the services

0.26996805111821087

In [40]:
global_mean = df_train_full['churn'].mean()
round(global_mean, 3)

0.27

In [41]:
categorical = ['gender', 'seniorcitizen', 'partner', 'dependents', 'phoneservice', 'multiplelines', 'internetservice',
              'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport', 'streamingtv', 'streamingmovies',
              'contract', 'paperlessbilling', 'paymentmethod']
numerical = ['tenure', 'monthlycharges', 'totalcharges']

In [42]:
df_train_full[categorical].nunique() #all the variables here have very few unique values

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

In [44]:
female_mean = df_train_full[df_train_full['gender'] == 'female']['churn'].mean()
female_mean

0.27682403433476394

In [45]:
male_mean = df_train_full[df_train_full['gender'] == 'male']['churn'].mean()
male_mean

0.2632135306553911

In [46]:
partner_yes = df_train_full[df_train_full['partner'] == 'yes']['churn'].mean()
partner_no = df_train_full[df_train_full['partner'] == 'no']['churn'].mean()
partner_yes, partner_no

(0.20503330866025166, 0.3298090040927694)